In [1]:
import tensorflow as tf

## 0.简介
全连接层的核心结构与感知机并没有多大差别。它在感知机的基础上,将不连续的阶跃激活函数换成了其它平滑连续可导的激活函数,并通过堆叠多个网络层来增强网络的表达能力。

通过替换感知机的激活函数,同时并行堆叠多个神经元来实现多输入、多输出的网络层结构。如下图所示:

![](https://github.com/zfhxi/Learn_tensorflow/blob/master/ch06-%E7%A5%9E%E7%BB%8F%E7%BD%91%E7%BB%9C/img/04.png?raw=true)

并行堆叠了 2 个神经元,即 2 个替换了激活函数的感知机,构成 3 输入节点、2 个输出节点的网络层。其中第一个输出节点的输出为:

$o_1=\sigma(w_{11}\cdot x_1+w_{21}\cdot x_2+w_{31}\cdot x_3+b_1)$

第二个节点的输出:

$o_2=\sigma(w_{12}\cdot x_1+w_{22}\cdot x_2+w_{32}\cdot x_3+b_2)$

输出向量为$o=[o_1,o_2]$。整个网络层可以通过矩阵关系式表达：

$[o_1\quad o_2]=[x_1\quad x_2\quad x_3]@\left[\begin{matrix}w_{11}&w_{12}\\w_{21}&w_{22}\\w_{31}&w_{32}\end{matrix}\right]+[b_1\quad b_2]$

即

$O=X@W+b$

其中输入矩阵$X$的shape定义为$[b,d_{in}]$，$b$为样本数量，$d_{in}$为输入节点数（样本特征长度）；权值矩阵$W$的shape定义为$[d_{in},d_{out}]$，$d_{out}$为输出节点数，偏置向量$b$的shape定义为$[d_{out}]$

考虑批量并行计算，例如2个样本，$x^{(1)}=[x^{(1)}_1,x^{(1)}_2,x^{(1)}_3]$，$x^{(2)}=[x^{(2)}_1,x^{(2)}_2,x^{(2)}_3]$，则可以方便地将上式推广到批量形式：

$
\left[\begin{matrix}o_1^{(1)}&o_2^{(1)}\\o_1^{(2)}&o_2^{(2)}\end{matrix}\right]
=\left[\begin{matrix}x_1^{(1)}&x_2^{(1)}&x_3^{(1)}\\x_1^{(2)}&x_2^{(2)}&x_3^{(2)}\end{matrix}\right]
@\left[\begin{matrix}w_{11}&w_{12}\\w_{21}&w_{22}\\w_{31}&w_{32}\end{matrix}\right]
+[b1\quad b2]
$

其中输出矩阵$O$包含了$b$个样本的输出特征，shape为$[b,d_{out}]$。由于每个输出节点与全部的输入节点相连接，这种网络层称为全连接层(Full-connected Layer)或稠密连接层(Dense Layer)

## 1.张量方式实现
定义好权值张量$W$和偏置张量$b$，并利用TensorFlow提供的批量矩阵相乘函数`tf.matmul()`即可完成网络层的计算。

下面假定:
* 输入矩阵X的shape为\[2,28*28\]，两个样本
* 权值矩阵W的shape为\[28*28,256\]
* 偏置向量b的shape为\[256\]
* 输出O的shape为\[2,256\]

实现如下：

In [2]:
x=tf.random.normal([2,28*28])
w1=tf.Variable(tf.random.truncated_normal([28*28,256],stddev=0.1))
b1=tf.Variable(tf.zeros([256]))
o1=tf.matmul(x,w1)+b1 # 线性变换
o1=tf.nn.relu(o1) # 激活函数

print(o1.shape)

(2, 256)


## 2.层方式实现
全连接层有封装好的层实现方式：

`layers.Dense(units,activation)`
* units: 指定输出节点数
* activation: 激活函数类型

输入节点数会根据第一次运算时的输入shape确定,同时根据输入、输出节点数自动创建并初始化权值张量W和偏置张量b,因此在新建类Dense实例时,并不会立即创建权值张量W和偏置张量b,而是需要调用`build`函数或者直接进行一次前向计算,才能完成网络参数的创建。

In [3]:
x=tf.random.normal([4,28*28])
from tensorflow.keras import layers
# 创建全连接层，指定输出节点数和激活函数
fc=layers.Dense(256,activation=tf.nn.relu)
h1=fc(x)
print(h1.shape)

(4, 256)


获取权值W和偏置张量b

In [4]:
W=fc.kernel
b=fc.bias
print(W.shape)
print(b.shape)

(784, 256)
(256,)


获取网络层待优化（待训练）的参数：

In [5]:
paramaters=fc.trainable_weights
print(paramaters)

[<tf.Variable 'dense/kernel:0' shape=(784, 256) dtype=float32, numpy=
array([[ 0.0737713 ,  0.03893939, -0.01091818, ...,  0.04434957,
         0.0282714 , -0.0280586 ],
       [-0.00280415, -0.02636383,  0.07079422, ..., -0.07259096,
         0.02353547,  0.07522345],
       [-0.01159611, -0.02654509, -0.07125218, ...,  0.02324271,
        -0.03804038, -0.01585384],
       ...,
       [-0.0557959 ,  0.03258117, -0.07514894, ...,  0.02536245,
         0.05416979,  0.01563413],
       [ 0.07392903,  0.04835364, -0.03015666, ...,  0.00992661,
         0.04085183, -0.00453697],
       [ 0.06462882, -0.03370189,  0.01819995, ..., -0.04521985,
        -0.06010333, -0.04727587]], dtype=float32)>, <tf.Variable 'dense/bias:0' shape=(256,) dtype=float32, numpy=
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0

如果希望获得所有参数（不仅包含待优化，也包含不需要优化的参数）

In [6]:
all_params=fc.variables
print(all_params)

[<tf.Variable 'dense/kernel:0' shape=(784, 256) dtype=float32, numpy=
array([[ 0.0737713 ,  0.03893939, -0.01091818, ...,  0.04434957,
         0.0282714 , -0.0280586 ],
       [-0.00280415, -0.02636383,  0.07079422, ..., -0.07259096,
         0.02353547,  0.07522345],
       [-0.01159611, -0.02654509, -0.07125218, ...,  0.02324271,
        -0.03804038, -0.01585384],
       ...,
       [-0.0557959 ,  0.03258117, -0.07514894, ...,  0.02536245,
         0.05416979,  0.01563413],
       [ 0.07392903,  0.04835364, -0.03015666, ...,  0.00992661,
         0.04085183, -0.00453697],
       [ 0.06462882, -0.03370189,  0.01819995, ..., -0.04521985,
        -0.06010333, -0.04727587]], dtype=float32)>, <tf.Variable 'dense/bias:0' shape=(256,) dtype=float32, numpy=
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0

利用网络层类对象进行前向计算时,只需要调用类的`__call__`方法即可,即写成`fc(x)`方式便可,它会自动调用类的`__call__`方法,在`__call__`方法中会自动调用`call`方法,这一设定由TensorFlow框架自动完成,因此用户只需要将网络层的前向计算逻辑实现在`call`方法中即可

In [ ]:
import os
pid=os.getpid()
!kill -9 $pid

